In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_parquet('./data/chargecurves_train.parquet')

In [3]:
def plot_timeseries(group):
    print(group['nominal_power'].iloc[0])
    plt.figure(figsize=(10, 6))

    # Create figure and primary y-axis
    fig, ax1 = plt.subplots(figsize=(10, 6))
    group_id = group['id'].iloc[0]
    

    # Extract time as HH:MM string
    group['time_str'] = group['timestamp'].dt.strftime('%H:%M')

    # Plot SOC on primary y-axis
    ax1.plot(group['time_str'], group['soc'], linestyle='-', color='b', label='SOC')
    ax1.set_xlabel('Time')
    ax1.set_ylabel('State of Charge (SOC)', color='b')
    ax1.tick_params(axis='y', labelcolor='b')

    # Create secondary y-axis for Power
    ax2 = ax1.twinx()
    ax2.plot(group['time_str'], group['power'], linestyle='-', color='r', label='Power')
    ax2.set_ylabel('Power', color='r')
    ax2.tick_params(axis='y', labelcolor='r')

    # Set x-ticks for readability
    N = max(len(group) // 10, 1)  # Show approx 10 labels, but at least 1
    ax1.set_xticks(group['time_str'].iloc[::N])

    # Improve visualization
    ax1.set_title(f'Group ID: {group_id}')
    ax1.grid(True, linestyle='--', alpha=0.5)

    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45)

    plt.show()


### Feature engineering

In [4]:
df['duration'] = df.groupby('id')['timestamp'].transform(lambda x: x.max() - x.min())

In [ ]:
df.loc[df['duration'] > '0 days 00:45:00']#.groupby('id').count()

In [ ]:
df.loc[df['duration'] > '0 days 01:59:00'].groupby('location_id').count()

In [7]:
df['total_charged'] = df.groupby('id')['soc'].transform(lambda x: x.max() - x.min())
df['charging_rate_kW'] = df['total_charged'] / (df['duration'].dt.total_seconds() / 3600)

### where nominal power is less than power

In [8]:
outliers = df.loc[df['power'] > df['nominal_power']].groupby(['id'])
outliers_ids = list(outliers.groups.keys())

### investigating the total charged 

In [ ]:
df.loc[df['total_charged'] > 80]

Checking for seasonal differences

In [ ]:
months = list(range(1, 13))
month_names = ["January", "February", "March", "April", "May", "June",
               "July", "August", "September", "October", "November", "December"]

avg_total_charged_by_month = []

for month in months:
    month_df = df.loc[df['timestamp'].dt.month == month]
    month_session_total = month_df.groupby('id')['total_charged'].max()
    avg_total_charged_month = month_session_total.mean()
    avg_total_charged_by_month.append(avg_total_charged_month)
    print(f'Month {month_names[month-1]} avg total charged: {avg_total_charged_month}')

plt.figure(figsize=(10, 6))
plt.bar(months, avg_total_charged_by_month)

plt.xticks(months, month_names, rotation=45)  # rotate labels if needed

plt.xlabel('Month')
plt.ylabel('Average Total Charged per session')
plt.title('Average Total Charged per Session by Month')
plt.tight_layout()
plt.show()


### Grouped by ID, where charging sessions are very short

In [11]:
grouped_df = df.groupby('id')

In [ ]:
grouped_df.get_group(139971).head(50)

In [ ]:
grouped_counts = grouped_df.count()
grouped_counts

In [14]:
short_sessions_df = grouped_counts.loc[grouped_counts['timestamp'] < 5]

In [ ]:
short_sessions_ids = list(short_sessions_df.index)
short_sessions_ids

In [ ]:
filtered_df = df[df['id'].isin(short_sessions_ids)]  

# Step 3: Group by 'location_id'
grouped_location_df = filtered_df.groupby('location_id')

# Step 4: Get the first few rows per group
grouped_location_df.count()


In [ ]:
short_sessions_df.groupby('location_id').count()

In [ ]:
grouped_df.get_group(short_sessions_ids[25]).head()

In [ ]:
plot_timeseries(grouped_df.get_group(short_sessions_ids[25]))

In [ ]:
plot_timeseries(outliers.get_group(outliers_ids[15]))

In [21]:
starts_df = df[df['sub_id'] == 0]

In [ ]:
starts_df.head(10)

In [ ]:
plt.figure(figsize=(10, 6))


In [ ]:
grouped_df.get_group(4)

In [ ]:
starts_df.loc[:, 'hour'] = starts_df['timestamp'].dt.hour

In [ ]:
starts_df

In [27]:
hour_df= starts_df.groupby('hour')

In [ ]:
hour_counts = starts_df['hour'].value_counts().sort_index()

plt.bar(hour_counts.index, hour_counts.values)
plt.xlabel('Hour of the Day')
plt.ylabel('Count')
plt.title('Distribution of Charging Start Times')
plt.xticks(range(24))
plt.show()

In [ ]:
starts_df.loc[:, 'day'] = starts_df['timestamp'].dt.dayofweek

In [ ]:
starts_df

In [ ]:
day_df= starts_df.groupby('day')
day_counts = starts_df['day'].value_counts().sort_index()

plt.bar(day_counts.index, day_counts.values)
plt.xlabel('Day of the Week')
plt.ylabel('Count')
plt.title('Distribution of Charging Start Times')
plt.show()